In [1]:
import wbgapi as wb
import pandas as pd

In [2]:
#Variables of interest
INDICATORS = {
    "SP.DYN.LE00.IN": "life_expectancy_birth",
    "SH.DYN.MORT": "under5_mortality_per_1000",
    "SP.ADO.TFRT": "adolescent_fertility_rate",
    "SP.DYN.TFRT.IN": "total_fertility_rate",
    "SP.POP.GROW": "population_growth",
    "SH.H2O.BASW.ZS": "basic_water_access_pct",
    "SH.STA.BASS.ZS": "basic_sanitation_access_pct",
    "SE.PRM.ENRR": "primary_school_enrollment",
    "SP.URB.TOTL.IN.ZS": "urban_population_pct",
    "NY.GDP.PCAP.KD": "gdp_per_capita_constant_usd"
}

In [3]:
# Choose our regions
ssa = set(wb.region.members("SSF"))  # Sub-Saharan Africa
sa = set(wb.region.members("SAS"))   # South Asia
filtered_countries = list(ssa | sa)  # Union of sets

In [ ]:
#Create df of countries and years
df = wb.data.DataFrame(
    list(INDICATORS.keys()),       # indicator codes
    economy=filtered_countries,    
    time=range(1990, 2021),
    labels=False,                  # keep codes to rename to descriptive names later
    numericTimeKeys=True           # years are integers
)

In [5]:
df.reset_index(inplace=True)

In [ ]:
# Melt the df so it's one row per country-year-indicator
df_long = df.melt(
    id_vars=['economy', 'series'],
    value_vars=list(range(1990, 2021)),
    var_name='Year',
    value_name='Value'
)

In [8]:
# Pivot so each indicator is a separate column
df_final = df_long.pivot_table(
    index=['economy', 'Year'],
    columns='series',
    values='Value'
).reset_index()

In [10]:
# Rename indicators to be their descriptive names
df_final.rename(columns=INDICATORS, inplace=True)
df_final.rename(columns={'economy': 'Country'}, inplace=True)

df_final

series,Country,Year,gdp_per_capita_constant_usd,primary_school_enrollment,under5_mortality_per_1000,basic_water_access_pct,basic_sanitation_access_pct,adolescent_fertility_rate,life_expectancy_birth,total_fertility_rate,population_growth,urban_population_pct
0,AGO,1990,2380.176243,81.085777,222.7,NaN,NaN,145.432,41.854,7.272,3.392403,37.144
1,AGO,1991,2324.369376,73.899437,222.5,NaN,NaN,148.019,43.812,7.208,3.359055,38.580
2,AGO,1992,2118.166545,70.755379,222.4,NaN,NaN,148.957,42.267,7.138,3.274142,40.039
3,AGO,1993,1559.517152,NaN,222.0,NaN,NaN,149.975,42.190,7.065,3.195594,41.511
4,AGO,1994,1529.994431,NaN,221.6,NaN,NaN,151.929,43.567,6.990,3.241689,43.000
...,...,...,...,...,...,...,...,...,...,...,...,...
1669,ZWE,2016,1377.639277,95.734095,56.9,64.468961,37.529191,100.849,59.760,3.828,1.388200,32.296
1670,ZWE,2017,1422.193460,94.334173,55.0,63.996627,36.941674,99.053,60.263,3.768,1.442854,32.237
1671,ZWE,2018,1471.394890,93.755202,52.3,63.538773,36.357160,98.612,60.906,3.744,1.487416,32.209
1672,ZWE,2019,1356.838211,93.427097,51.1,63.094954,35.774336,98.733,61.060,3.748,1.563534,32.210


In [ ]:
#Save as CSV
df_final.to_csv("qtm350_final_data.csv", index=False)


In [ ]:
#Save as db
import sqlite3

# Create a connection to a new database file
conn = sqlite3.connect("worldbank_data.db")

# Write the DataFrame to a table called "indicators"
df_final.to_sql("indicators", conn, if_exists="replace", index=False)

# Close the connection
conn.close()